In [1]:
-- Add media type handler for `text/html` requests
CREATE DOMAIN "text/html" AS TEXT;

CREATE DOMAIN

Total execution time: 00:00:00.001

In [2]:
-- Sanitize text to replace characters with HTML entities
CREATE OR REPLACE FUNCTION ft.sanitize_html(text) RETURNS text AS $$
  SELECT REPLACE(REPLACE(REPLACE(REPLACE(REPLACE($1, '&', '&amp;'), '"', '&quot;'),'>', '&gt;'),'<', '&lt;'), '''', '&apos;')
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.003

In [55]:
-- Format all cars as HTML cards
CREATE OR REPLACE FUNCTION ft.html_cards(ft.vehicle) RETURNS text AS $$
  WITH json_data AS (
    SELECT
      key, value
    FROM 
      jsonb_each_text($1.data)
  ),
  html_table AS (
    SELECT 
      string_agg(
        FORMAT(
          '<tr><td>%s</td><td>%s</td></tr>',
          json_data.key, json_data.value
        ),
        ''
      ) as table_rows
    FROM json_data
  )
  SELECT FORMAT($html$
    <div class="card">
      <div class="card-body">
        <h5 class="card-title">%2$s</h5>
        <p class="card-text">
          <table class="table">
            <tbody>
              %4$s
            </tbody>
          </table>
        </p>
      </div>
    </div>
  $html$,
  $1.id,
  ft.sanitize_html($1.name),
  $1.data,
  (SELECT table_rows FROM html_table)
  );
$$ LANGUAGE sql STABLE;


CREATE FUNCTION

Total execution time: 00:00:00.005

In [56]:
-- Create HTML to display all Vehicles
-- DROP FUNCTION IF EXISTS ft.html_all_vehicle(text);
CREATE OR REPLACE FUNCTION ft.html_all_vehicle(text) RETURNS text AS $$
  SELECT COALESCE(
    '<div class="card-columns">'
      || string_agg(ft.html_cards(n), '' ORDER BY n.id) ||
    '</div>'
    '<p class="">'||$1||'</p>',
    '<p class="">No Vehicle.</p>'
    '<p class="">'||$1||'</p>'
  )
  FROM ft.vehicle n where $1 = 'all_vehicle' OR data ->> 'type'=$1 ;
$$ language sql;


CREATE FUNCTION

Total execution time: 00:00:00.002

In [ ]:
-- add filter to filter vehichle based on type
CREATE OR REPLACE

In [57]:

-- Generate page to display Vehicles
CREATE OR REPLACE FUNCTION ft.index() RETURNS "text/html" AS $$
  SELECT $html$
  <!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Postgres Full stack</title>
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
</head>
<body>
  <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
    <a class="navbar-brand" href="/rpc/index">Fullstack postgres vehicle app</a>
    <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNav"
      aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <div class="collapse navbar-collapse" id="navbarNav">
      <ul class="navbar-nav">
        <li class="nav-item active">
          <a class="nav-link" href="/rpc/index">Vehicles</a>
        </li>
        <li class="nav-item">
          <a class="nav-link" href="/rpc/new">Create Vehicle</a>
        </li>
      </ul>
    </div>
  </nav>

  <div class="container mt-4">
    <h2>Vehicles</h2>
    
    <div class="form-group">
      <label for="vehicleType">Select Vehicle Type:</label>
      <select class="form-control" id="vehicleType" onchange="updateVehicles()">
        <option value="all_vehicle">All Vehicle</option>
        <option value="car">Car</option>
        <option value="bike">Bike</option>
        <option value="truck">Truck</option>
      </select>
    </div>

    <div id="VehiclesContent">
      <!-- Initial content will be for 'car' -->
      <div id="Vehicles">
        $html$
          || ft.html_all_vehicle('all_vehicle') ||
        $html$
      </div>
    </div>
  </div>

  <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/@popperjs/core@2.9.2/dist/umd/popper.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
  <script>
  function updateVehicles() {
    var vehicleType = document.getElementById('vehicleType').value;
    var vehicleHTML = '';
    switch(vehicleType) {
      case 'car':
        vehicleHTML = `
          $html$
            || ft.html_all_vehicle('car') ||
          $html$
        `;
        break;
      case 'bike':
        vehicleHTML = `
          $html$
            || ft.html_all_vehicle('bike') ||
          $html$
        `;
        break;
      case 'truck':
        vehicleHTML = `
          $html$
            || ft.html_all_vehicle('truck') ||
          $html$
        `;
        break;
       case 'all_vehicle':
        vehicleHTML = `
          $html$
            || ft.html_all_vehicle('all_vehicle') ||
          $html$
        `;
        break;
      default:
        vehicleHTML = '<p>No Vehicles found.</p>';
    }
    document.getElementById('Vehicles').innerHTML = vehicleHTML;
  }
</script>

</body>
</html>

  $html$
$$ language sql;


CREATE FUNCTION

Total execution time: 00:00:00.010